In [1]:
# Access the API through environment variable
import os
#from dotenv import load_dotenv
#load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SQLDatabase
from llama_index.readers.wikipedia import WikipediaReader

In [4]:
from sqlalchemy import (
    create_engine,
    MetaData,
    ForeignKey,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

from sqlalchemy.orm import declarative_base, relationship

In [5]:
engine = create_engine("sqlite:///:memory:", future=True)
metadata_obj = MetaData()

In [6]:
# create city SQL table
table_name = "product"
product_table = Table(
    table_name,
    metadata_obj,
    Column("sku", Integer, primary_key=True),
    Column("name", String(100), nullable=False),
    Column("product_category", String(30), nullable=False),
    Column("measurement_type", String(50), nullable=False),
    Column("cable_type", String(30), nullable=False),
    extend_existing=True,
)

table_name = "specs"
specs_table = Table(
    table_name,
    metadata_obj,
    Column("id", Integer, primary_key=True),  # Auto-incrementing ID
    Column("sku", Integer, ForeignKey("product.sku")),
    Column("spec_value", String(200)),
    Column("spec_name", String(200)),
    extend_existing=True,
)

metadata_obj.create_all(engine)

In [7]:
# print tables
metadata_obj.tables.keys()

dict_keys(['product', 'specs'])

In [8]:
import pandas as pd
from sqlalchemy import create_engine
from pathlib import Path

# Assuming 'engine' is already created and connected to your in-memory SQLite database

# Define the paths to your Excel files
base_path = Path('lmc_specs')  # Adjust this if your notebook's working directory is different
product_table_path = base_path / 'product_table.xlsx'
spec_table_path = base_path / 'spec_table.xlsx'

# Read the Excel files into DataFrames
product_df = pd.read_excel(product_table_path)
spec_df = pd.read_excel(spec_table_path)

# Load the DataFrames into the corresponding tables in the database
# If the tables already exist, append the data. Ensure you're using the same 'engine' as before
product_df.to_sql('product', con=engine, if_exists='append', index=False, method='multi')
spec_df.to_sql('specs', con=engine, if_exists='append', index=False, method='multi')

print("Data loaded into the database successfully.")


Data loaded into the database successfully.


In [9]:
sql_database = SQLDatabase(engine, include_tables=["product", "specs"])

In [12]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

In [10]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")

In [13]:
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["product", "specs"],
    llm=llm
)

In [14]:
query_str = "What are the specs for a PM10?"
response = sql_query_engine.query(query_str)

INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'product' has columns: sku (INTEGER), name (VARCHAR(100)), product_category (VARCHAR(30)), measurement_type (VARCHAR(50)), cable_type (VARCHAR(30)), and foreign keys: .

Table 'specs' has columns: id (INTEGER), sku (INTEGER), spec_value (VARCHAR(200)), spec_name (VARCHAR(200)), and foreign keys: ['sku'] -> product.['sku'].
> Table desc str: Table 'product' has columns: sku (INTEGER), name (VARCHAR(100)), product_category (VARCHAR(30)), measurement_type (VARCHAR(50)), cable_type (VARCHAR(30)), and foreign keys: .

Table 'specs' has columns: id (INTEGER), sku (INTEGER), spec_value (VARCHAR(200)), spec_name (VARCHAR(200)), and foreign keys: ['sku'] -> product.['sku'].
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "

In [16]:
from IPython.display import Markdown, display
display(Markdown(f"{response}"))

The specs for a PM10 laser include a maximum energy density of 600 millijoules per square centimeter, a minimum wavelength of 250 nanometers, a maximum wavelength of 11000 nanometers, a minimum power of 0.01 watts, a maximum power of 10 watts, and a cooling type of air. The wavelength range is from 0.25 to 11 micrometers, with a power range of 10 milliwatts to 10 watts. Other specs include a maximum intermittent power of 30 watts for less than 5 minutes, a resolution of 1 milliwatt, and a maximum power density of 6 kilowatts per square centimeter. The PM10 is compatible with various meters and has a cable length of 2 meters.